In [ ]:
import numpy as np
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

class DecisionTree:
    def  __init__(self, max_depth=5, min_samples_split=2, min_samples_leaf=5, random_state=0):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.random_state = random_state #seed
        self.tree = None   # tree is a dictionary 

    def fit(self, X, y):
        #tbp
        #döntési fa tanítása
        return None
    
    def predict(self, X):
        #tbp
        #regresszios ertek meghatarozasa
        return None
    
    def print():
        #döntési fa ábrázolása
        #tbp
        return None

In [ ]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

#Minta adatforrás
wine = load_wine()
X = wine.data # adatok
y = wine.target # címkék/célváltozók

#Adatok felosztása tanító és tesztelő halmazra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) # 80% tanító, 20% tesztelő

#Döntési fa tanítása
dt = DecisionTree()
dt.fit(X_train, y_train)

#Döntési fa kiértékelése
y_pred = dt.predict(X_test)
print(y_pred)

dt.print()